In [ ]:
%matplotlib qt
import mne
import os
from mne.preprocessing import ICA

In [ ]:
raw_fname=('raw_fif/')#define raw file name
subjectlist=os.listdir(raw_fname)#get each subject name

In [ ]:
for subject in subjectlist[0:1]:#loop with subject
    montage=mne.channels.read_montage('standard_1020')#read montage
    raw_name_path=os.path.join(raw_fname,subject)#Concatenate file name 
    raw=mne.io.read_raw_fif(raw_name_path,montage,preload=True)
    raw.set_eeg_reference(ref_channels='average',projection=False)#re-reference with all channel averaged
    raw.filter(0.5,None,fir_design='firwin')#high pass filter
    raw.filter(None,30,fir_design='firwin')#low pass filter
    events,events_id=mne.events_from_annotations(raw,event_id='auto')#read events from raw annotations
    event_id = {'No_Bad':10,'No_Good':12,'No_Normal':14,'Yes_Bad':11,'Yes_Good':13,'Yes_Normal':15}#define events and events_id
    events = mne.pick_events(events, include=[10,12,14,11,13,15])#pick events that we interested
    epochs = mne.Epochs(raw,events,event_id=event_id,baseline=(None,0),preload=True,tmin=-0.3, tmax=0.55)#epoch and baseline correction
    ica = ICA(n_components=63, method='fastica').fit(epochs)#run ica based on epochs data with fastica method 
    ica.save('Epochs_ica/'+subject[0:5] +'-ica.fif')#save ICA and epochs data
    epochs.save('Epochs/'+subject[0:5] +'-epo.fif')